In [1]:
import pandas as pd
import os
import re

In [2]:
def sort_num(el):
    num = re.search(r"version_01_(\w+)_(\d+)", el)
    return int(num.group(2))
FOLDER = './cooked_df_k2/'
START_INDEX = 1
END_INDEX = -1

In [3]:
l = os.listdir(FOLDER)
parquets = [el for el in l if el.endswith('.parquet')]
csvs = [el for el in l if el.endswith('.csv')]
parquets.sort(key=sort_num)
csvs.sort(key=sort_num)

In [6]:
df_values_loaded = pd.read_parquet(FOLDER + 'df_version_01_value_1.parquet', engine="fastparquet")
df_ids_loaded = pd.read_csv(FOLDER + 'df_version_01_id_1.csv')
df_trunk = pd.concat([df_ids_loaded, df_values_loaded.T.reset_index(drop=True)], axis=1)
df_trunk.set_index(['K2ID','PLANET_NUM'], inplace=True)
df_trunk.head()

,,LABEL,0,1,2,3,4,5,6,7,8,...,155544,155545,155546,155547,155548,155549,155550,155551,155552,155553
K2ID,PLANET_NUM,,,,,,,,,,,,,,,,,,,,,
WASP-151,1,0,0.986693,0.986002,0.987026,0.987246,0.986427,0.986373,0.988111,0.987127,0.987296,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K2-270,1,0,0.965709,0.965980,0.964331,0.967006,0.970852,0.949135,0.970174,0.965442,0.968827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPIC 220198551,1,1,0.945357,0.957158,0.955344,0.952642,0.955580,0.942663,0.958278,0.959890,0.946657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K2-187,1,0,0.971182,0.973194,0.973903,0.976708,0.961849,0.980099,0.970986,0.972030,0.982274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPIC 211897272,1,1,0.994646,0.994345,0.994204,0.993822,0.993556,0.993477,0.993338,0.993037,0.992869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for pair in zip(csvs[START_INDEX:END_INDEX], parquets[START_INDEX:END_INDEX]):
    df_values_loaded = pd.read_parquet(FOLDER + pair[1], engine="fastparquet")
    df_ids_loaded = pd.read_csv(FOLDER + pair[0])
    df_full = pd.concat([df_ids_loaded, df_values_loaded.T.reset_index(drop=True)], axis=1)
    df_full = df_full.set_index(['KEPID', 'PLANET_NUM'])
    print(df_full)

    df_trunk = pd.concat((df_trunk, df_full), axis=0)
df_trunk.shape

(500, 155555)

In [9]:
df_trunk.tail()

,,LABEL,0,1,2,3,4,5,6,7,8,...,155544,155545,155546,155547,155548,155549,155550,155551,155552,155553
K2ID,PLANET_NUM,,,,,,,,,,,,,,,,,,,,,
EPIC 205040048,1,1,0.978494,0.978141,0.977342,0.978443,0.978151,0.980721,0.979913,0.982115,0.982980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPIC 220436943,1,1,0.986072,0.989351,0.990561,0.992879,0.991937,0.992138,0.991261,0.990898,0.990175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K2-276,1,0,0.994274,0.994730,0.994263,0.994624,0.994422,0.994872,0.994710,0.994055,0.994449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPIC 229004835,2,0,0.999426,0.999489,0.999546,0.999544,0.999572,0.999569,0.999663,0.999553,0.999595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HIP 41378,1,0,0.997920,0.998229,0.998333,0.998477,0.998475,0.998562,0.998613,0.998613,0.998603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_trunk['LABEL'].value_counts()

LABEL
1    270
0    169
2     61
Name: count, dtype: int64

In [11]:
#{"CONFIRMED": 0, "CANDIDATE": 1, "FALSE POSITIVE": 2}
# marked candidates as confirmed to simulate robovetter. may change in the future
df_trunk.loc[df_trunk['LABEL'] == 1, 'LABEL'] = 0

In [12]:
df_trunk['LABEL'].value_counts()

LABEL
0    439
2     61
Name: count, dtype: int64